**Carregando o córpus**

In [2]:
import nltk
nltk.download('names')

from nltk.corpus import names
import random

nomes_masculinos = [(nome, 'masc') for nome in names.words('male.txt')]
nomes_femininos = [(nome, 'fem') for nome in names.words('female.txt')]

nomes = nomes_masculinos + nomes_femininos 
random.shuffle(nomes)

for nome in nomes[:10]:
  print(nome)

('Dick', 'masc')
('Kattie', 'fem')
('Jakob', 'masc')
('Ellwood', 'masc')
('Peggie', 'fem')
('Lillian', 'fem')
('Welch', 'masc')
('Bartlett', 'masc')
('Chevy', 'masc')
('Bartholemy', 'masc')


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


**Extração de Features**

In [9]:
def extrair_features(nome):
  caracter2id = {c:i for i, c in enumerate('abcdefghijklmnopqrstuyvxwz_')}

  try:
    primeiro_caractere = caracter2id[nome.lower()[0]]
  except:
    primeiro_caractere = caracter2id['_']

  try:
    ultimo_caractere = caracter2id[nome.lower()[-1]]
  except:
    ultimo_caractere = caracter2id['_']

  total_caractere = len(nome)

  return [primeiro_caractere, ultimo_caractere, total_caractere]

X, y = [], []
for row in nomes[:10]:
  nome, categoria = row

  features = extrair_features(nome)

  classe = 1 if categoria == 'fem' else 0
  print(nome, features, classe)

  X.append(features)
  y.append(classe)

Dick [3, 10, 4] 0
Kattie [10, 4, 6] 1
Jakob [9, 1, 5] 0
Ellwood [4, 3, 7] 0
Peggie [15, 4, 6] 1
Lillian [11, 13, 7] 1
Welch [24, 7, 5] 0
Bartlett [1, 19, 8] 0
Chevy [2, 21, 5] 0
Bartholemy [1, 21, 10] 0


**Árvore de Decisão**

In [12]:
from sklearn.tree import DecisionTreeClassifier

ad = DecisionTreeClassifier()
ad.fit(X,y)

DecisionTreeClassifier()

In [13]:
nome = 'Maria'
features = extrair_features(nome)

previsao = ad.predict([features])

if previsao == 1:
  print('Feminino')
else:
  print('Masculino')

Feminino


**Naive Bayes**

In [14]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X, y)

GaussianNB()

In [15]:
nome = 'Tiago'
features = extrair_features(nome)

previsao = nb.predict([features])

if previsao == 1:
  print('Feminino')
else:
  print('Masculino')

Masculino


**Regressão Logística**

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X,y)

LogisticRegression()

In [17]:
nome = 'Francisco'
features = extrair_features(nome)

previsao = lr.predict([features])

if previsao == 1:
  print('Feminino')
else:
  print('Masculino')

Masculino


**Avaliação**

In [18]:
tam = int(len(nomes) * 0.2)
treino = nomes[tam:]
teste = nomes[:tam]

len(treino), len(teste)

(6356, 1588)

In [19]:
treino_X = [extrair_features(row[0]) for row in treino]
treino_y = [1 if row[1] == 'fem' else 0 for row in treino]

ad = DecisionTreeClassifier()
ad.fit(treino_X, treino_y)

nb = GaussianNB()
nb.fit(treino_X, treino_y)

lr = LogisticRegression()
lr.fit(treino_X, treino_y)

LogisticRegression()

In [20]:
teste_x = [extrair_features(row[0]) for row in teste]
teste_y = [1 if row[1] == 'fem' else 0 for row in teste]

predicao_ad = ad.predict(teste_x)

predicao_nb = nb.predict(teste_x)

predicao_lr = lr.predict(teste_x)

In [21]:
from sklearn.metrics import accuracy_score, f1_score

acuracia = accuracy_score(teste_y, predicao_ad)
f1 = f1_score(teste_y, predicao_ad)
print('Arvore de Decisão - Acurácia: ', round(acuracia, 2), 'F1: ', round(f1, 2))

acuracia = accuracy_score(teste_y, predicao_nb)
f1 = f1_score(teste_y, predicao_nb)
print('Naive Bayes - Acurácia: ', round(acuracia, 2), 'F1: ', round(f1, 2))

acuracia = accuracy_score(teste_y, predicao_lr)
f1 = f1_score(teste_y, predicao_lr)
print('Arvore de Decisão - Acurácia: ', round(acuracia, 2), 'F1: ', round(f1, 2))


Arvore de Decisão - Acurácia:  0.74 F1:  0.8
Naive Bayes - Acurácia:  0.7 F1:  0.78
Arvore de Decisão - Acurácia:  0.69 F1:  0.77


**Exemplo Real**

Objetivo de treinar um classificador para prever se uma revisão é boa, neutra ou ruim. Utilizando como córpus as revisões de produtos da B2W Digital, uma empresa de comércio eletrônico.

**Baixando o córpus**

In [22]:
!wget https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/master/B2W-Reviews01.csv

--2023-04-23 23:14:38--  https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/master/B2W-Reviews01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49453175 (47M) [text/plain]
Saving to: ‘B2W-Reviews01.csv’

B2W-Reviews01.csv   100%[===================>]  47.16M   251MB/s    in 0.2s    

2023-04-23 23:14:41 (251 MB/s) - ‘B2W-Reviews01.csv’ saved [49453175/49453175]



**Carregando o córpus**

In [33]:
import csv

with open('B2W-Reviews01.csv') as f:
  reader = csv.reader(f, delimiter=',', quotechar='\"')
  corpus = list(reader)

  header, corpus = corpus[0], corpus[1:]

random.shuffle(corpus)

In [34]:
header

['submission_date',
 'reviewer_id',
 'product_id',
 'product_name',
 'product_brand',
 'site_category_lv1',
 'site_category_lv2',
 'review_title',
 'overall_rating',
 'recommend_to_a_friend',
 'review_text',
 'reviewer_birth_year',
 'reviewer_gender',
 'reviewer_state']

In [37]:
for row in corpus[0]:
  print(row)

2018-01-02 12:45:20
86e1bd9c028c285f53a47dc92cedbfa2d15470235aae947e8e6a6762671c3632
22469906
Kit Proteção Para Moto G5 5" Capa Transparente + Película De Vidro

Celulares e Smartphones
Acessórios para Celular
Gostei muito do produto
5
Yes
Produto excelente gostei muito e recomendo> Produto muito bom
1976.0
F
ES


**Separando o córpus em conjuntos de treinamento e teste**

In [39]:
size = int(len(corpus) * 0.2)
treino = corpus[size:]
teste = corpus[:size]

len(treino), len(teste)

(105899, 26474)

**Extraindo as features**

In [40]:
def get_features(info):
	product_id = info[2]
	product_name = info[3]
	product_brand = info[4]
	site_category_lv1 = info[5]
	site_category_lv2 = info[6]
	review_title = info[7]
	overall_rating = info[8]
	recommend_to_a_friend = info[9]
	review_text = info[10]
	reviewer_birth_year = info[11]
	reviewer_gender = info[12]
	reviewer_state = info[13]

	x = ' '.join([product_name, 
		      product_brand,
	 	      site_category_lv1,
		      site_category_lv2,
		      review_title,
		      review_text])
	y = 2 if overall_rating in ['4', '5'] else 0 if overall_rating in ['1', '2'] else 1
	return x, y

**Representação Vetorial TF-IDF**

In [43]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

def tokenize(texto):
	return nltk.word_tokenize(texto)

pipe = Pipeline([
	('count', CountVectorizer(tokenizer=tokenize, stop_words=stopwords, min_df=5)),
	('tfidf', TfidfTransformer()),
	('regression', LogisticRegression(max_iter=1000))
])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Treinando o modelo**

In [44]:
feat_treino = [get_features(w) for w in treino]
treino_x = [w[0] for w in feat_treino]
treino_y = [w[1] for w in feat_treino]

pipe.fit(treino_x, treino_y)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('count',
                 CountVectorizer(min_df=5,
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...],
                                 tokenizer=<function tokenize at 0x7fcd422251f0>)),
                ('tfidf', TfidfTransformer()),
                ('regression', LogisticRegression(max_iter=1000))])

**Testando o Modelo**

In [45]:
feat_teste = [get_features(w) for w in teste]
teste_X = [w[0] for w in feat_teste]
teste_y = [w[1] for w in feat_teste]

prediction = pipe.predict(teste_X)

**Avaliando o desempenho**

In [47]:
acuracia = accuracy_score(teste_y, prediction)
f1 = f1_score(teste_y, prediction, average='weighted')
print('Regressão Logística - Acurácia:', round(acuracia, 2), '/ F1-Score:', round(f1, 2))

Regressão Logística - Acurácia: 0.85 / F1-Score: 0.82
